In [1]:
import pandas as pd

C:\Users\Ashish Singh\AppData\Local\Temp\ipykernel_8968\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
data = pd.read_excel(
    r"D:\OpenHack\Hackathon Datasets\Crop Yield Data\APY_Potato.xls")

In [14]:
data.head()

,Place,Date,Season,Area,Production,Yield
4,ADILABAD,2000-01,Whole Year,18,68,3.777778
5,ADILABAD,2005-06,Whole Year,2,17,8.5
6,ADILABAD,2006-07,Whole Year,3,45,15
7,ANANTAPUR,2000-01,Whole Year,4,34,8.5
8,ANANTAPUR,2002-03,Whole Year,2,17,8.5


In [6]:
# remove first 2 rows and make the 3rd row as header
data = data[2:]

In [7]:
# in first column fill the NaN values with the previous value
data.iloc[:, 0] = data.iloc[:, 0].fillna(method='ffill')

C:\Users\Ashish Singh\AppData\Local\Temp\ipykernel_8968\1808912994.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.iloc[:, 0] = data.iloc[:, 0].fillna(method='ffill')


In [8]:
# forward fill the second column
data.iloc[:, 1] = data.iloc[:, 1].fillna(method='ffill')

C:\Users\Ashish Singh\AppData\Local\Temp\ipykernel_8968\2741510801.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.iloc[:, 1] = data.iloc[:, 1].fillna(method='ffill')


In [9]:
# delete the rows where 3rd column has NaN values
data = data.dropna(subset=[data.columns[2]])

In [11]:
# rename the columns
data.columns = ['Place', 'Date', 'Season', 'Area', 'Production', 'Yield']

In [12]:
# remove first 2 characters from the 'Place' column
data['Place'] = data['Place'].str[2:]

In [13]:
# remove first character from the place column (cuz only a digit and . was visible)
data['Place'] = data['Place'].str[1:]

In [15]:
# remove . from the 'Place' column (inspecting the the other rows in the tail of the dataframe, there was again .)
data['Place'] = data['Place'].str.replace('.', '')

In [16]:
# remove digits from the 'Place' column
data['Place'] = data['Place'].str.replace(r'\d+', '', regex=True)

In [ ]:
import pandas as pd
import requests

import time

cleaned_data = data

# get the unique places
places = cleaned_data['Place'].unique()

# make a list for storing the coordinates
coordinates_list = []

# for each place get the coordinates
for place in places:
    url = f"https://geocode.maps.co/search?q={place}&api_key=65cfa174890c9160700920myo18621b"
    time.sleep(0.5)
    response = requests.get(url)
    try:
        data = response.json()
        data = data[0]
        lat = data['lat']
        lon = data['lon']
        # the coordinates are to be save as rounded to x.00 or x.25 or x.50 or x.75
        lat = round(float(lat) * 4) / 4
        lon = round(float(lon) * 4) / 4
        coordinates_list.append([place, lat, lon])
        print(f"Coordinates for {place} are {lat}, {lon}")
    except:
        print(f"Error getting coordinates for {place}")

# add the coordinates to the dataframe
coordinates = pd.DataFrame(coordinates_list, columns=['Place', 'Latitude', 'Longitude'])

In [ ]:
import pandas as pd

combined_data = pd.read_csv('combined_data.csv')

# get the unique latitudes and longitudes combinations
lat_lon = combined_data[['Latitude', 'Longitude']].drop_duplicates()

# for each latitude and longitude in the combined data, get the place by comparing the latitude and longitude with the coordinates in the coordinates data
places = []
for index, row in lat_lon.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    # if place is found, add it to the list of places
    if len(coordinates[(coordinates['Latitude'] == lat) & (coordinates['Longitude'] == lon)]) > 0:
        place = coordinates[(coordinates['Latitude'] == lat) & (coordinates['Longitude'] == lon)]['Place'].values[0]
        places.append(place)
    else:
        places.append('Unknown')
# add the places to the combined data
lat_lon['Place'] = places

In [ ]:
combined_data = pd.read_csv('combined_data.csv')
coordinates = lat_lon

In [ ]:
combined_data.shape

In [ ]:
coordinates.shape

In [ ]:
# join the two dataframes on longitude and latitude
joined_data = combined_data.join(coordinates.set_index(
    ['Longitude', 'Latitude']), on=['Longitude', 'Latitude'])

In [ ]:
joined_data.head()

In [ ]:
joined_data.shape

In [ ]:
joined_data.value_counts('Place')

In [ ]:
# drop the rows with unknown place
joined_data = joined_data[joined_data['Place'] != 'Unknown']

In [ ]:
joined_data.shape

In [ ]:
joined_data.head()

In [ ]:
production_data = data

In [ ]:
joined_data.info()

In [ ]:
production_data.info()

In [ ]:
joined_data.shape

In [ ]:
production_data['Season'].value_counts()

In [ ]:
# delete the rows where the season value is Total
production_data = production_data[production_data['Season'] != 'Total']

In [ ]:
# make another column in combined_data to store the year of the date
joined_data['Year'] = pd.DatetimeIndex(joined_data['Date']).year

In [ ]:
# first extract first 4 characters of the date
production_data['Year'] = production_data['Date'].astype(str).str[:5]

In [ ]:
# convert the year of production data to integer
production_data['Year'] = production_data['Year'].astype(int)

In [ ]:
# join the two dataframes on the year column and place column
merged_data = pd.merge(joined_data, production_data, on=['Year', 'Place'])

In [ ]:
merged_data.shape

In [ ]:
df = merged_data

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# change format of Date_x which is in string format DD-MM-YYYY to datetime
df['Date_x'] = pd.to_datetime(df['Date_x'], format='%d-%m-%Y')

In [ ]:
# drop Date_y column
df = df.drop(columns=['Date_y'])

In [ ]:
# trim white spaces from Season column
df['Season'] = df['Season'].str.strip()

In [ ]:
print(df['Season'].unique())

In [ ]:
# for a place and year where the season value is Kharif, delete the rows where the date_x's month is 11, 12, 1, 2, 3, 4, 5
df = df[~((df['Season'] == 'Kharif') & (
    df['Date_x'].dt.month.isin([11, 12, 1, 2, 3, 4, 5])))]

# for a place and year where the season value is Rabi, delete the rows where the date_x's month is 4 5 6 7 8 9
df = df[~((df['Season'] == 'Rabi') & (
    df['Date_x'].dt.month.isin([4, 5, 6, 7, 8, 9])))]

# for a place and year where the season value is winter, delete the rows where the date_x's month is 3 4 5 6 7 8 9 10 11
df = df[~((df['Season'] == 'Winter') & (
    df['Date_x'].dt.month.isin([3, 4, 5, 6, 7, 8, 9, 10, 11])))]

# for a place and year where the season value is summer, delete the rows where the date_x's month is 7, 8, 9, 10, 11, 12, 1, 2, 3
df = df[~((df['Season'] == 'Summer') & (
    df['Date_x'].dt.month.isin([7, 8, 9, 10, 11, 12, 1, 2, 3])))]

In [ ]:
df.shape

In [ ]:
# drop Latitude, Longitude, Area, Yield
df = df.drop(['Latitude', 'Longitude', 'Yield', 'Date_x'], axis=1)

In [ ]:
# Group by the data by Place, Year, Season and take the mean of the values
df2 = df.groupby(['Place', 'Year', 'Season']).mean().reset_index()

In [ ]:
# plot graph between year and other weather parameters
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.figure(figsize=(20, 10))
plt.subplot(2, 2, 1)
sns.lineplot(x='Year', y='Temperature', data=df2)
plt.title('Year vs Temperature')

plt.subplot(2, 2, 2)
sns.lineplot(x='Year', y='Relative_humidity', data=df2)
plt.title('Year vs Humidity')

plt.subplot(2, 2, 3)
sns.lineplot(x='Year', y='Surface_net_solar_radiation', data=df2)
plt.title('Year vs radiaion')

plt.subplot(2, 2, 4)
sns.lineplot(x='Year', y='Total_precipitation', data=df2)
plt.title('Year vs rain')

plt.show()

In [ ]:
# save the data to a csv file
df2.to_csv('Onion_production_data.csv', index=False)